<a href="https://colab.research.google.com/github/Algebraistponey/NLPworkshop/blob/main/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!wget https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/data/labeled_data.csv -O hate_speech_data.csv
# !wget https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/data/labeled_data.p -O hate_speech_data.p


--2021-03-25 17:59:09--  https://github.com/t-davidson/hate-speech-and-offensive-language/blob/master/data/labeled_data.csv
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘hate_speech_data.csv’

hate_speech_data.cs     [ <=>                ] 104.53K  --.-KB/s    in 0.04s   

2021-03-25 17:59:10 (2.79 MB/s) - ‘hate_speech_data.csv’ saved [107041]



In [ ]:
!pip install torch==1.6.0
!pip install torchtext==0.7.0

     |████████████████████████████████| 748.8MB 22kB/s 
ERROR: torchvision 0.9.0+cu101 has requirement torch==1.8.0, but you'll have torch 1.6.0 which is incompatible.
ERROR: torchtext 0.9.0 has requirement torch==1.8.0, but you'll have torch 1.6.0 which is incompatible.
  Found existing installation: torch 1.8.0+cu101
    Uninstalling torch-1.8.0+cu101:
      Successfully uninstalled torch-1.8.0+cu101
     |████████████████████████████████| 4.5MB 5.3MB/s 
     |████████████████████████████████| 1.2MB 38.8MB/s 
  Found existing installation: torchtext 0.9.0
    Uninstalling torchtext-0.9.0:
      Successfully uninstalled torchtext-0.9.0


In [40]:
import random

import os

from datetime import datetime

#deal with tensors
import torch   

#handling text data
from torchtext import data 

import pandas as pd

#for attention LSTM
from torch.autograd import Variable

In [43]:

def tokenizer(text): # create a tokenizer function
    return text.split(' ')

TWEET = data.Field(tokenize=tokenizer,batch_first=True,include_lengths=True)

CLASS = data.LabelField(dtype = torch.float, batch_first=True)

training_data=data.TabularDataset(path = 'hate_speech_data.csv',
                                  format = 'csv',
                                  fields = [
                                      (None, None),
                                      ('tweet', TWEET),
                                      ('class', CLASS)
                                  ],
                                  skip_header = True)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Ple

In [41]:
print(vars(training_data.examples[0])

SyntaxError: ignored

In [44]:
#Reproducing same results
SEED = 2315

#Torch
torch.manual_seed(SEED)

#Cuda algorithms
torch.backends.cudnn.deterministic = True

In [26]:
train_data, test_data = training_data.split(split_ratio=0.5, random_state = random.seed(SEED))
train_data, valid_data = train_data.split(split_ratio=0.4, random_state = random.seed(SEED))

In [37]:
TEXT.build_vocab(train_data, min_freq=3, vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

In [ ]:

print("Size of TEXT vocabulary: {}\n".format(len(TEXT.vocab)))

print("Size of LABEL vocabulary: {}\n".format(len(LABEL.vocab)))

print("Commonly used words: {}\n".format(TEXT.vocab.freqs.most_common(10)))


In [34]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')  

#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, test_iterator, valid_iterator = data.BucketIterator.splits(
    (train_data, test_data, valid_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    device = device)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [33]:
import torch.nn as nn

class classifier(nn.Module):
    
    #define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout):
        
        #Constructor
        super().__init__()          
        
        #embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        #lstm layer
        self.lstm = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout,   #dense layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        #activation function
        self.act = nn.Sigmoid()
        
    def forward(self, text, text_lengths):
        
        #text = [batch size,sent_length]
        embedded = self.embedding(text)
        #embedded = [batch size, sent_len, emb dim]
      
        #packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,batch_first=True)
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
        
        #concat the final forward and backward hidden state
        hidden = torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)
                
        #hidden = [batch size, hid dim * num directions]
        dense_outputs=self.fc(hidden)

        #Final activation function
        outputs=self.act(dense_outputs)
        
        return outputs

SyntaxError: ignored